In [1]:
!pip install -r requirements.txt

In [1]:
from persistence import Connection as Database
db: Database = Database()

Connected to DB


In [2]:
resumes = db.fetch_all_unlabeled_resumes()

In [3]:
len(resumes)

30

In [4]:
resumes[8]

{'_id': ObjectId('63747aa2514a492b64bbd9d6'),
 'title': 'Data Scientist at SAP',
 'contactInfo': {'address': 'Abraham Lincoln Street 1, Chicago, IL',
  'website': 'my-website.com',
  'linkedin': 'linkedin.com/in/me123',
  'name': 'Monica Music',
  'phoneNumber': '01012345',
  'email': 'monica@music.co.kr',
  'github': 'monicamusic123',
  'birthday': '24 December 1985',
  'family': '5 kids, divorced'},
 'summary': {'value': 'Tomas is dedicated, persistent and organized. He excels in communication and loves problem-solving in team environments.',
  'label': 0},
 'experience': {'value': [{'value': {'company': 'Tesla',
     'title': 'Data Scientist',
     'location': 'San Francisco Bay Area',
     'fromDate': 'July 2021',
     'toDate': 'Present',
     'description': ''},
    'label': 0},
   {'value': {'company': 'Tesla',
     'title': 'Robotics Deep ML Researcher',
     'location': 'Remote',
     'fromDate': 'July 2020',
     'toDate': 'December 2020',
     'description': 'Trained MaskRCN

In [5]:
resume = resumes[11]

In [6]:
import importlib

In [7]:
job_description = ""
with open("jd_examples/SE_JD2.txt", 'r') as f:
    job_description = f.read()
    

In [8]:
from model import Job
importlib.reload(Job)
Job = Job.Job
from model.keyword_utils import extract_requirements_JD

from persistence.cloud_storage.cloud_storage import read_job_keywords
from util.json_to_df import create_df, get_resume_dict_from_dataframe

job_title = "Data Scientist"
description_keywords = extract_requirements_JD(job_description)
job_keywords_csv = read_job_keywords(job_title)
job = Job(job_title)
job.load_from_csv_data(job_keywords_csv)
job.update_keywords_from_description(description_keywords)


resume_df, md = create_df(resume)

In [9]:
from model import resume_scorer
# Make sure to reload everytime
importlib.reload(resume_scorer)
ResumeScorer = resume_scorer.ResumeScorer

scorer: ResumeScorer = ResumeScorer(job)
scored = scorer.score_resume_as_dataframe(resume_df)


Closest keyword to 'english' is 'start-up environment' with cosine similarity 0.3315032378943554



/opt/conda/lib/python3.7/site-packages/gensim/models/keyedvectors.py:1166: RuntimeWarning: invalid value encountered in true_divide
  similarities = dot_products / (norm * all_norms)


In [10]:
set(scored['keywords'].map(lambda x: [v[0] for v in x]).sum())

{'academic',
 'academic called',
 'academic mobility',
 'account actual',
 'acess mssql',
 'adei advanced',
 'advanced data',
 'ajax',
 'allow',
 'analyze results',
 'angular java',
 'apache',
 'apache cordova',
 'application software',
 'applications patients',
 'approaches',
 'approaches mysql',
 'architectural design',
 'architecture net',
 'authorization called',
 'babel',
 'babel django',
 'bachelor',
 'bachelor science',
 'backbone',
 'based dynamic',
 'ble',
 'ble apache',
 'bluetooth ble',
 'bonn',
 'boot',
 'browsing',
 'business',
 'calculate cost',
 'called',
 'called fiduswriter',
 'cassandra',
 'children',
 'children discuss',
 'club',
 'club organized',
 'collaboration',
 'collaboration editor',
 'com',
 'com javascript',
 'communication management',
 'company',
 'company automatically',
 'computer',
 'computer operator',
 'computer science',
 'condition equipment',
 'cost repairs',
 'council',
 'council tpu',
 'created',
 'created data',
 'created module',
 'cultures',
 

In [11]:
importlib.reload(resume_scorer)

cut, stats = resume_scorer.shorten_resume(scored, description_keywords)

print(len(scored))
print(len(cut))
cut

Space used: {'body': 1768, 'skill': 41, 'summary': 0}
Total space: {'body': 1806, 'skill': 210, 'summary': 120}
21
16


,value,label,type,address,website,linkedin,name,phoneNumber,email,github,...,scoring_text,multiplier,keywords,raw_keywords,education_heuristic,experience_heuristic,interest_heuristic,covered_keywords,keyword_cover_heuristic,score
8,NaN,0.141783,education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Master of Science (M.Sc.), Computer Science",1.00,"[(computer science, 0.7522), (master science, ...","[computer science, master science, science sc,...",inf,0.0,0.0,[],0.0,inf
9,NaN,0.138283,education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Bachelor of Science (B.Sc.), Information Techn...",1.00,"[(bachelor science, 0.7377), (bachelor, 0.5829...","[bachelor science, bachelor, science sc, sc in...",inf,0.0,0.0,[],0.0,inf
10,NaN,0.250181,education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,computer operator,1.00,"[(computer operator, 1.0), (operator, 0.73), (...","[computer operator, operator, computer]",inf,0.0,0.0,[],0.0,inf
5,NaN,0.059389,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Software Developer EAM system is the applicati...,1.00,"[(eam application, 0.6897), (repair eam, 0.585...","[eam application, repair eam, application soft...",0.0,inf,0.0,[],0.0,inf
3,NaN,0.089667,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Software Developer http://www.polycare-project...,1.00,"[(node js, 0.493), (software developer, 0.4901...","[node js, software developer, java, developer ...",0.0,inf,0.0,[java],0.5,inf
4,NaN,0.112727,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Software Developer Development of the online c...,1.00,"[(collaboration editor, 0.6519), (online colla...","[collaboration editor, online collaboration, d...",0.0,inf,0.0,"[python, javascript, development]",1.5,inf
18,NaN,0.720000,hardSkills,NaN,NaN,NaN,Java,NaN,NaN,NaN,...,Java,0.80,"[(java, 1.0)]",[java],0.0,0.0,0.0,[],0.0,0.720000
17,NaN,0.720000,hardSkills,NaN,NaN,NaN,JavaScript,NaN,NaN,NaN,...,JavaScript,0.80,"[(javascript, 1.0)]",[javascript],0.0,0.0,0.0,[],0.0,0.720000
6,NaN,0.068438,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Technician, Software Engineer Project 1: Docum...",1.00,"[(qt oracle, 0.4858), (oracle, 0.4757), (workf...","[qt oracle, oracle, workflow jsc, editor jsc, ...",0.0,0.0,0.0,[sql],0.5,0.568438
2,NaN,0.457302,experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Software Engineer,1.00,"[(software engineer, 1.0), (engineer, 0.741), ...","[software engineer, engineer, software]",0.0,0.0,0.0,[],0.0,0.457302


In [12]:
desc = ""
for res in resumes: 
    # summary = res['summary']
    for exp in res['education']['value']:
        v = exp['value']['description']
        if len(v) > 200:
            desc = v
            break

In [13]:
from gensim.models import Word2Vec
import numpy as np

In [14]:
model = Word2Vec.load("model/embedding/model_skill_extraction")
word_vec = model.wv

In [15]:
embeddings = []
for kw in job.keywords:
    kw = kw.lower()
    try: 
        embedding = word_vec[kw]
        embeddings.append(embedding)
    except: 
        parts = kw.split(" ")
        for part in parts:
            emb = np.zeros(300)
            try: 
                sub = word_vec[part]
                emb += sub
            except: 
                print(f"Could not find part: {part}")
        embeddings.append(emb)


Could not find part: predictive
Could not find part: tensorflow
Could not find part: predictive
Could not find part: artificial
Could not find part: scikit-learn
Could not find part: apache
Could not find part: spark
Could not find part: hadoop
Could not find part: git
Could not find part: hive
Could not find part: scala
Could not find part: pandas
Could not find part: numpy
Could not find part: mathmatics
Could not find part: scrum
Could not find part: telematics
Could not find part: four‐year
Could not find part: devops
Could not find part: start-up
Could not find part: language/system
Could not find part: core/greengrass
Could not find part: cloud”
Could not find part: telematics
Could not find part: wi-fi


In [16]:
python = word_vec['python']
sims = word_vec.cosine_similarities(python, embeddings)
sims = sims[~np.isnan(sims)]
closest = np.argsort(sims)[::-1]

for close in closest: 
    print(f"{list(job.keywords.keys())[close]}: {sims[close]}")


python: 1.000000013306145
four‐year: 0.6615131996685364
sql: 0.6511003981173904
hadoop: 0.5073278857002472
big data: 0.4934231344058615
economics: 0.45766625906178754
data analytics: 0.4350928301126456
mathematics: 0.4186828204570275
tableau: 0.4147991999048025
git: 0.3979078328480121
machine learning: 0.39508272428639657
deep learning: 0.39508272428639657
device software development: 0.37459524909399994
environment: 0.3386189679169399
data mining: 0.33543237797717085
tensorflow: 0.33543237797717085
spark: 0.33543237797717085
hive: 0.3160992802460906
scikit-learn: 0.2991629513012904
predictive modeling: 0.279409576316896
statistical modeling: 0.279409576316896
corporate: 0.2614567282510001
contemporary software development approaches: 0.25998505852570614
concept: 0.2500107271204663
data visualization: 0.24701394099436158
r: 0.2425657797910773
artificial intelligence: 0.22084526250921788
statistics: 0.21799325151685156
software engineering: 0.21048095995911634
it field: 0.20429192371773

/opt/conda/lib/python3.7/site-packages/gensim/models/keyedvectors.py:1166: RuntimeWarning: invalid value encountered in true_divide
  similarities = dot_products / (norm * all_norms)
